<a href="https://colab.research.google.com/github/BrianSward/colab-work/blob/main/extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook goes and fetches a website, extracts the text content, formats it into a more digestable format, and returns this information to the user. 

This will bypass things like:
- ads 
- paywalls *(assuming the text is delivered / an immediate log-in is not required)*

It also removes **ALL NON TEXT**, for good or bad. So there will be no pictures, videos, etc.

This function was taken from my Jupyter Notebook [here](https://github.com/BrianSward/colab-work/blob/main/fake_news_machine.ipynb). I wanted to see what was being returned from the scraping portion of my machine learning notebook and check for any strange artifacts or anomalies that could alter the accuracy of my model.


In [ ]:
!pip install trafilatura

In [7]:
# imports for the web scraping

from bs4 import BeautifulSoup
import json
import numpy as np
import requests
import trafilatura
import textwrap

In [12]:
def beautifulsoup_extract_text_fallback(response_content):
    """
    Extract text from HTML using BeautifulSoup.
    """
    soup = BeautifulSoup(response_content, 'html.parser')
    text = soup.find_all(text=True)
    blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script', 'style']
    cleaned_text = ' '.join(item for item in text if item.parent.name not in blacklist).replace('\t', '')
    return cleaned_text.strip()

def extract_text_from_single_web_page(url):
    """
    Extract text from a web page using Trafilatura or BeautifulSoup.
    """
    downloaded_url = trafilatura.fetch_url(url)
    try:
        extracted_data = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True, include_comments=False,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        extracted_data = trafilatura.extract(downloaded_url, json_output=True, with_metadata=True,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    if extracted_data:
        json_output = json.loads(extracted_data)
        return json_output['text']
    else:
        try:
            resp = requests.get(url)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except requests.exceptions.MissingSchema:
            return np.nan

# Get input URL from user
user_url = input("Provide me a URL: ")
print()

try:
    # Extract text from the web page
    page_text = extract_text_from_single_web_page(url=user_url)

    # Split the text into paragraphs and wrap each paragraph to a maximum of 80 characters per line
    paragraphs = str(page_text).split("\n")
    for paragraph in paragraphs:
        wrapped_paragraph = textwrap.wrap(paragraph, width=80)
        # Print the wrapped paragraph lines
        for line in wrapped_paragraph:
            print(line)
        # Print a blank line to separate paragraphs
        print()
except ValueError:
    print("Invalid URL entered.")

Provide me a URL: https://www.mlive.com/life/2023/04/willie-nelson-to-celebrate-his-90th-birthday-on-the-road-in-michigan-in-2023.html

CLARKSTON, MI - Willie Nelson is going on the road again. This time, to
celebrate his 90th birthday. The living legend will bring his Outlaw Music
Festival tour to Pine Knob Music Theatre on Friday, September 22.

Also in concert will be Bobby Weir & Wolf Bros. featuring The Wolfpack, The
String Cheese Incident and Particle Kid. Tickets go on sale Friday, April 28 at
10 a.m.

Nelson said he wanted to celebrate his milestone 90th birthday, which is on
April 29, on the road with his friends, family and fans.

“I am so thrilled to announce these additional dates for our 2023 Outlaw Music
Festival Tour,” said Nelson. “I can’t wait to keep the celebration of my 90th
birthday going into the fall with this great lineup of artists, my friends and
family, and of course, the amazing fans.”

RELATED: Every concert coming to Pine Knob for its 2023 summer concert s